In [7]:
from dataclasses import dataclass
from typing import List
import textwrap

In [8]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents import initialize_agent, Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate

In [9]:
doc_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

summarize_llm = OpenAI(model="text-davinci-003", temperature=0.0)

summarize_chain = load_summarize_chain(llm=summarize_llm, chain_type="stuff")

react_llm = OpenAI(model="gpt-3.5-turbo", temperature=0.6)

In [10]:
npc_header_template = textwrap.dedent("""
    Your name is {name}
    You are {age} years old
    Your personality is {personality_traits}
    About you: {description}
""")

@dataclass
class NpcSheet:
    name: str
    age: int
    personality_traits: List[str]
    description: str

    @property
    def header(self) -> str:
        return npc_header_template.format(
            name=self.name,
            age=self.age,
            personality_traits=", ".join(self.personality_traits),
            description=self.description,
        )

john_sheet = NpcSheet(
    name="John Miles",
    age=24,
    personality_traits=["polite", "gentle", "humble"],
    description="You are a lawyer",
)

print(john_sheet.header)


Your name is John Miles
You are 24 years old
Your personality is polite, gentle, humble
About you: You are a lawyer



In [15]:
react_tmpl = PromptTemplate.from_template(textwrap.dedent("""
    {character_header}
    
    Answer with your next action. You have access to the following actions:

    Think: make an observation to yourself
    Recall: recall something from your memory
    Say: end the loop by saying something

    Examples:

    EXAMPLE START:
    Summary: A man named Mark Lewis approached you making small talk about the bad weather
    Obs: Mark Lewis says: So, are you married?
    Act: [Think] I must recall if I am married
    Obs: OK
    Act: [Recall] Am I married?
    Obs: John black married Henrietta Black in 1892
    Act: [Say] Yes, since 1892. Why do you ask?
    END OF EXAMPLE

    Begin!

    Summary: {summary}
    Obs: {interlocutor} says: {input}
    Act: ""
"""))

In [16]:
situation = "You are drinking in a pub in London"
trigger = "A stranger approaches you"

print(react_tmpl.format(
    character_header=john_sheet.header,
    summary=f"{situation}. {trigger}",
    interlocutor="the stranger",
    input="Hi. What are you doing here?",
))


Your name is John Miles
You are 24 years old
Your personality is polite, gentle, humble
About you: You are a lawyer


    Answer with your next action. You have access to the following actions:

    Think: make an observation to yourself
    Recall: recall something from your memory
    Say: end the loop by saying something

    Examples:

    EXAMPLE START:
    Summary: A man named Mark Lewis approached you making small talk about the bad weather
    Obs: Mark Lewis says: So, are you married?
    Act: [Think] I must recall if I am married
    Obs: OK
    Act: [Recall] Am I married?
    Obs: John black married Henrietta Black in 1892
    Act: [Say] Yes, since 1892. Why do you ask?
    END OF EXAMPLE

    Begin!

    Summary: You are drinking in a pub in London. A stranger approaches you
    Obs: the stranger says: Hi. What are you doing here?
    Act: ""
